An excellent formal treatment from https://liorpachter.wordpress.com/2013/09/21/the-needleman-wunsch-algorithm/<br>
Note that here, a 'match' is regarded as a semantically 'positive' event, such that 'mismatch' and 'space' get negative scores. In that case, the recursive function uses a max. The other way round, where 'match' is a '0' and mismatches and gaps get positive numbers, the recursive function uses a min.

In [1]:
import numpy as np

In [16]:
def NW_mxs(a, b, method='bool', m=1, x=-1, s=-1):
    """Take 2 strings, a and b, of lengths n1 and n2
    And compute NW edit distance between them
    Compute a matrix S of shape n1+1 by n2+1, 
    and recursively fill it.
    
    Params
    ------
    a, b: input strings
    m: int, match score
    x: int, mismatch score
    s: int, 'gap' score
    
    Returns
    -------
    H: edit distance matrix
    tr: traceback matrix
    """
    n1 = len(a)
    n2 = len(b)
    H = np.zeros([n1+1,n2+1])
    dt = np.dtype("i4, i4")
    tb = np.zeros([n1+1,n2+1], dtype=dt)

    for i in range(1,n1+1):
        H[i][0] = i*s
    for j in range(1,n2+1):
        H[0][j] = j*s
    
    if method == 'bool':
        for i in range(1, n1+1):
            for j in range(1, n2+1):
                H[i][j] = max(H[i-1][j] + s, 
                              H[i][j-1] + s,
                              H[i-1][j-1] + x*is_match(a[i-1], b[j-1]))
                #Fill in a traceback matrix
                #In case of a tie, priority is given to a match.
                if H[i][j] == H[i-1][j-1] + x*is_match(a[i-1], b[j-1]):
                    tb[i][j] = (-1, -1)
                elif H[i][j] == H[i-1][j] + s:
                    tb[i][j] = (-1, 0)
                elif H[i][j] == H[i][j-1] + s:
                    tb[i][j] = (0, -1)
                    
    elif method == 'match_mismatch':
        for i in range(1, n1+1):
            for j in range(1, n2+1):
                H[i][j] = max(H[i-1][j] + s, 
                              H[i][j-1] + s,
                              H[i-1][j-1] + _match_mismatch(a[i-1],b[j-1],m,x))
                #Fill in a traceback matrix
                #In case of a tie, priority is given to a match.
                if H[i][j] == (H[i-1][j-1] + _match_mismatch(a[i-1],b[j-1],m,x)):
                    tb[i][j] = (-1, -1)
                elif H[i][j] == H[i-1][j] + s:
                    tb[i][j] = (-1, 0)
                elif H[i][j] == H[i][j-1] + s:
                    tb[i][j] = (0, -1)
    
    return H, tb

def NW_match_mismatch(a, b, m=1, x=-1, s=-1):
    """Take 2 strings, a and b, of lengths n1 and n2
    And compute NW edit distance between them
    Compute a matrix S of shape n1+1 by n2+1, 
    and recursively fill it.
    
    Params
    ------
    a, b: input strings
    m: int, match score
    x: int, mismatch score
    s: int, 'gap' score
    
    Returns
    -------
    H: edit distance matrix
    tr: traceback matrix
    """
    n1 = len(a)
    n2 = len(b)
    H = np.zeros([n1+1,n2+1])
    dt = np.dtype("i4, i4")
    tb = np.zeros([n1+1,n2+1], dtype=dt)

    for i in range(1,n1+1):
        H[i][0] = i*s
    for j in range(1,n2+1):
        H[0][j] = j*s
                    
    for i in range(1, n1+1):
        for j in range(1, n2+1):
            H[i][j] = max(H[i-1][j] + s, 
                          H[i][j-1] + s,
                          H[i-1][j-1] + _match_mismatch(a[i-1],b[j-1],m,x))
            #Fill in a traceback matrix
            #In case of a tie, priority is given to a match.
            if H[i][j] == (H[i-1][j-1] + _match_mismatch(a[i-1],b[j-1],m,x)):
                tb[i][j] = (-1, -1)
            elif H[i][j] == H[i-1][j] + s:
                tb[i][j] = (-1, 0)
            elif H[i][j] == H[i][j-1] + s:
                tb[i][j] = (0, -1)
    
    return H, tb


def SW(a, b, m=1, s=-1, x=-1):
    n1 = len(a)
    n2 = len(b)
    H = np.zeros([n1+1,n2+1])
    dt = np.dtype("i4, i4")
    tb = np.zeros([n1+1,n2+1], dtype=dt)

    for i in range(1,n1+1):
        H[i][0] = 0
    for j in range(1,n2+1):
        H[0][j] = 0
        
    for i in range(1, n1+1):
        for j in range(1, n2+1):
            H[i][j] = max(H[i-1][j] + s, 
                          H[i][j-1] + s,
                          H[i-1][j-1] + _match_mismatch(a[i-1],b[j-1],m,x),
                         0)
            #Fill in a traceback matrix
            #In case of a tie, priority is given to a match.
            if H[i][j] == (H[i-1][j-1] + _match_mismatch(a[i-1],b[j-1],m,x)):
                tb[i][j] = (-1, -1)
            elif H[i][j] == H[i-1][j] + s:
                tb[i][j] = (-1, 0)
            elif H[i][j] == H[i][j-1] + s:
                tb[i][j] = (0, -1)
    
    return H


def NW_kt(a, b, m=0, ins=1, dl=1, s=1, verbose=False):
    n1 = len(a)
    n2 = len(b)
    H = np.zeros([n1+1,n2+1])
    dt = np.dtype("i4, i4")
    tb = np.zeros([n1+1,n2+1], dtype=dt)
    
    if verbose:
        print("{m, i, d, s} = (%s, %s, %s, %s)"% (m, ins, dl, s))

    for i in range(1,n1+1):
        H[i][0] = i*ins
    for j in range(1,n2+1):
        H[0][j] = j*dl
        
    for i in range(1, n1+1):
        for j in range(1, n2+1):
            H[i][j] = min(H[i-1][j] + dl, 
                          H[i][j-1] + ins,
                          H[i-1][j-1] + _match_mismatch(a[i-1],b[j-1],m,s))
            #Fill in a traceback matrix
            #In case of a tie, priority is given to a match.
            if H[i][j] == (H[i-1][j-1] + _match_mismatch(a[i-1],b[j-1],m,s)):
                tb[i][j] = (-1, -1)
            elif H[i][j] == H[i-1][j] + dl:
                tb[i][j] = (-1, 0)
            elif H[i][j] == H[i][j-1] + ins:
                tb[i][j] = (0, -1)
    
    return H


def is_match(x1,x2):
    """returns 1 if x1 == x2, 0 otherwise"""
    match = 0
    if x1 == x2:
        match = 1
        
    return match

def _match_mismatch(x1, x2, m, x):
    """Returns m if x1 == x2, x otherwise"""
    score = 0
    if x1 == x2:
        score = m
    else:
        score = x
    return score

In [3]:
a = 'gaattacaa'
b = 'gattacaa'

m = 1 #match score
x = -1 #mismatch score
s = -1 #gap

H, tr = NW_mxs(a,b,method='bool')
H2, tr2 = NW_match_mismatch(a,b)

In [4]:
H #similarity function s(a,b): 1 if the letters a == b, 0 otherwise. 

array([[ 0., -1., -2., -3., -4., -5., -6., -7., -8.],
       [-1., -1., -1., -2., -3., -4., -5., -6., -7.],
       [-2., -1., -2., -1., -2., -3., -4., -5., -6.],
       [-3., -2., -2., -2., -1., -2., -3., -4., -5.],
       [-4., -3., -2., -3., -2., -1., -2., -3., -4.],
       [-5., -4., -3., -3., -3., -2., -1., -2., -3.],
       [-6., -5., -4., -3., -3., -3., -2., -2., -3.],
       [-7., -6., -5., -4., -3., -3., -3., -2., -2.],
       [-8., -7., -6., -5., -4., -4., -3., -3., -3.],
       [-9., -8., -7., -6., -5., -5., -4., -4., -4.]])

In [5]:
H2 #similarity function s(a,b): 1 if the letters a == b, -1 otherwise. 

array([[ 0., -1., -2., -3., -4., -5., -6., -7., -8.],
       [-1.,  1.,  0., -1., -2., -3., -4., -5., -6.],
       [-2.,  0.,  2.,  1.,  0., -1., -2., -3., -4.],
       [-3., -1.,  1.,  1.,  0.,  1.,  0., -1., -2.],
       [-4., -2.,  0.,  2.,  2.,  1.,  0., -1., -2.],
       [-5., -3., -1.,  1.,  3.,  2.,  1.,  0., -1.],
       [-6., -4., -2.,  0.,  2.,  4.,  3.,  2.,  1.],
       [-7., -5., -3., -1.,  1.,  3.,  5.,  4.,  3.],
       [-8., -6., -4., -2.,  0.,  2.,  4.,  6.,  5.],
       [-9., -7., -5., -3., -1.,  1.,  3.,  5.,  7.]])

In [6]:
a = 'xyabcyz'
b = 'prabcrs'
H3 = SW(a,b)

In [7]:
H3

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  2.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  3.,  2.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  2.,  2.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.]])

In [17]:
a = 'bed'
b = 'bead'
NW_kt(a,b,ins=5, verbose=True)

{m, i, d, s} = (0, 5, 1, 1)


array([[  0.,   1.,   2.,   3.,   4.],
       [  5.,   0.,   2.,   3.,   4.],
       [ 10.,   1.,   0.,   3.,   4.],
       [ 15.,   2.,   1.,   1.,   3.]])